In [1]:
import pandas as pd
import numpy as np
import data_modifications as dm
from gluonts.time_feature import time_features_from_frequency_str


In [2]:
tickers = ['WFC', 'AAPL', 'MA', 'BAC', 'GOOGL', 'AMZN', 'JPM', 'MSFT', 'V',
       'ABBV', 'NVDA', 'LLY', 'MRK', 'UNH', 'JNJ']

In [3]:
df = dm.core_frame('AAPL')
df = dm.encoded_source(df)
df['rob_sentiment'] = df['rob_sentiment'].apply(dm.sentiment_to_value)
df['encoded_sentiment'] = df['rob_sentiment'] * df['Source']
sums = df.groupby('stock_time')['encoded_sentiment'].apply(lambda x: np.sum(np.stack(x), axis =0))
df = pd.merge(df, sums, left_on = 'stock_time', right_index = True)
df[~df.apply(lambda row: np.array_equal(row['encoded_sentiment_x'], row['encoded_sentiment_y']), axis=1)]['encoded_sentiment_y'].iloc[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0])

In [4]:
time_features = time_features_from_frequency_str('15min')
time_names = {0:'minute_of_hour', 1:'hour_of_day', 2:'day_of_week', 3:'day_of_month', 4:'day_of_year'}
time_features

[<function gluonts.time_feature._base.minute_of_hour(index: pandas.core.indexes.period.PeriodIndex) -> numpy.ndarray>,
 <function gluonts.time_feature._base.hour_of_day(index: pandas.core.indexes.period.PeriodIndex) -> numpy.ndarray>,
 <function gluonts.time_feature._base.day_of_week(index: pandas.core.indexes.period.PeriodIndex) -> numpy.ndarray>,
 <function gluonts.time_feature._base.day_of_month(index: pandas.core.indexes.period.PeriodIndex) -> numpy.ndarray>,
 <function gluonts.time_feature._base.day_of_year(index: pandas.core.indexes.period.PeriodIndex) -> numpy.ndarray>]

In [13]:
for tick in tickers:
    df = dm.core_frame(tick)
    df = dm.encoded_source(df)
    df['rob_sentiment'] = df['rob_sentiment'].apply(dm.sentiment_to_value)
    df['total_articles'] = df['Publishing Time'].notna().astype(int)
    df['total_articles'] = df.groupby('stock_time')['total_articles'].transform(sum)
    df['encoded_sentiment'] = df['rob_sentiment'] * df['Source']
    sums = df.groupby('stock_time')['encoded_sentiment'].apply(lambda x: np.sum(np.stack(x), axis =0))
    df = pd.merge(df, sums, left_on = 'stock_time', right_index = True)
    df.drop(columns=['Publishing Time','Source', 'rob_sentiment', 'encoded_sentiment_x'], inplace = True)
    df.rename(columns = {'encoded_sentiment_y':'encoded_sentiment'}, inplace = True)
    df['encoded_sentiment'] = df['encoded_sentiment'].apply(lambda x: tuple(x))
    # print(tick, df[df['total_articles']>1]['encoded_sentiment'].iloc[5])
    # print(df.duplicated().sum())
    df.drop_duplicates(inplace =True, ignore_index=True)
    for i,function in enumerate(time_features):
        df['stock_time'] = df['stock_time'].apply(lambda x: pd.Period(x, freq = '15min'))
        df[time_names[i]] = df['stock_time'].apply(function)
    df.to_csv('./csv_files/'+tick+'_core.csv', index = False)

 

/var/folders/tg/nkrt27q937bgzsnrzszxn7bc0000gn/T/ipykernel_70704/528971735.py:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df['total_articles'] = df.groupby('stock_time')['total_articles'].transform(sum)
/var/folders/tg/nkrt27q937bgzsnrzszxn7bc0000gn/T/ipykernel_70704/528971735.py:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df['total_articles'] = df.groupby('stock_time')['total_articles'].transform(sum)
/var/folders/tg/nkrt27q937bgzsnrzszxn7bc0000gn/T/ipykernel_70704/528971735.py:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas,

In [12]:
df = pd.read_csv('./csv_files/WFC_core.csv')
df.tail(30)

,Unnamed: 0,stock_time,open,high,low,close,volume,numtrades,vwap,total_articles,encoded_sentiment,minute_of_hour,hour_of_day,day_of_week,day_of_month,day_of_year
33848,33848,2024-03-14 19:30,57.2500,57.2900,57.120,57.2500,1248706,9777,57.216509,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.008475,0.326087,0.000000,-0.066667,-0.30000
33849,33849,2024-03-14 19:45,57.2550,57.4300,57.150,57.3900,2783191,17785,57.312098,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.262712,0.326087,0.000000,-0.066667,-0.30000
33850,33850,2024-03-14 20:00,57.3910,57.4000,57.370,57.4000,4901286,91,57.370013,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-0.500000,0.369565,0.000000,-0.066667,-0.30000
33851,33851,2024-03-15 13:30,56.8600,58.1600,56.860,57.9100,3546979,9070,57.245192,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.008475,0.065217,0.166667,-0.033333,-0.29726
33852,33852,2024-03-15 13:45,57.9200,58.0950,57.750,58.0500,574743,6326,57.935688,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.262712,0.065217,0.166667,-0.033333,-0.29726
33853,33853,2024-03-15 14:00,58.0500,58.2650,57.880,58.2550,648260,5473,58.054415,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-0.500000,0.108696,0.166667,-0.033333,-0.29726
33854,33854,2024-03-15 14:15,58.2500,58.3000,58.040,58.1850,649580,5620,58.168575,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-0.245763,0.108696,0.166667,-0.033333,-0.29726
33855,33855,2024-03-15 14:30,58.1890,58.1900,58.010,58.0800,633465,5510,58.099510,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.008475,0.108696,0.166667,-0.033333,-0.29726
33856,33856,2024-03-15 14:45,58.0600,58.2442,58.020,58.0850,611049,4856,58.139315,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.262712,0.108696,0.166667,-0.033333,-0.29726
33857,33857,2024-03-15 15:00,58.0900,58.1750,57.990,57.9975,746406,4502,58.085200,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-0.500000,0.152174,0.166667,-0.033333,-0.29726
